In [1]:
import csv
from pathlib import Path
import datetime

In [2]:
import time

In [3]:
import pickle
import shutil
import warnings

### Configクラス 

In [4]:
class Config():
    """
    各クラス・関数が参照する変数をまとめたクラス
    """
    def __init__(self):
        self.parent = None

In [5]:
config = Config()

### 各種クロージャ 

#### enableクロージャ 

CounterClosierの親の作成・処理を行う．

In [6]:
class MultiClosier():
    """
    複数のカウンタを利用する時に，このクロージャの終了時にまとめてtempfileを削除するためのクロージャ
    """
    def __init__(self, parent):
        self.parent = parent
        
    def __enter__(self):
        if config.parent is not None:
            raise Exception("MultiCount has already opend. cannot open another MultiCount")
        
        config.parent = self.parent
        return self
        
    def __exit__(self, ex_type, ex_value, trace):
        config.parent = None  # 共通して行う
        if ex_type is None:# 正常終了した場合
            self.parent.all_close()
        return False

#### counterクロージャ 

プログラムのメイン．一時ファイル・保存ファイルの処理を行う．

In [7]:
class CounterClosier():
    """
    イテレーションの進捗をtempfileに保存するクロージャ，自身によってイテレータをラップする．
    途中で例外によって終了した場合と親が存在する場合にファイルを残す．
    """
    def __init__(self, file_path, parent, each_save=False, save_span=1):
        """
        file_path: pathlib.Path
            一時ファイルのパス
        parent: ParentCounter
            自身の親を意味するクラス
        each_save: bool
            指定回数ごとに保存するかどうか
        save_span: int, default:1
            指定価数ごとに保存する場合の，指定回数
        """
        self.file_path = file_path
        self.parent = parent
        self.each_save = each_save
        self.save_span = save_span
        
        # 保存オブジェクト・保存関数の初期化
        self.object = None
        self.object_path = None
        self.load_funcs = []
        self.save_funcs = []
        self.func_paths = []
        self.src_dsts = []
        
        
        # 一時ファイルの読み込み
        self.start_counter = self._read_tempfile()
        self.counter = 0  # 一応こちらでも0に初期化
        
    def _read_tempfile(self):
        """
        一時ファイルの読み込み
        """
        if self.file_path.exists():
            with open(self.file_path, "r") as f:
                reader = csv.reader(f)
                #dateについて取得, 現在時間との差が一日以内かどうか判定
                datetime_list = next(reader)  # [datetime,実際の日時の文字列]
                tempfile_datetime = datetime.datetime.strptime(datetime_list[1], "%Y-%m-%d %H:%M:%S")
                if datetime.datetime.now() - tempfile_datetime >= datetime.timedelta(days=1):
                    warnings.warn("tempfile is not recent date, please check tempfile")

                # スタートカウンターの読み込み
                start_counter_list = next(reader)
                start_counter = int(start_counter_list[1])
        else:
            start_counter = 0
        
        return start_counter
    
    def _load_object(self):
        """
        オブジェクト保存ファイルの読み込み
        """
        new_object = self.object  # とりあえず現在のオブジェクトとする
        
        if self.object_path is not None:
            if self.object_path.exists():
                with open(self.object_path, "rb") as f:
                    new_object = pickle.load(f)
                    
        return new_object
    
    def _save_object(self):
        """
        オブジェクト保存ファイルの書き出し
        """
        if self.object is not None:
            with open(self.object_path, "wb") as f:
                pickle.dump(self.object, f)
                
    def save_load_object(self, obj, obj_path):
        """
        オブジェクトの保存についての設定と読み込み．保存ファイルが存在しなかった場合は，引数のオブジェクトがそのまま返る．
        obj: any
            保存するオブジェクト
        obj_path: path
            保存するパス
        """
        self.object = obj
        self.object_path = obj_path
        
        # オブジェクト保存ファイルの読み込み
        self.object = self._load_object()
        
        return self.object
    
    def _load_funcs(self):
        """
        ロード関数による読み込み
        """
        for func_path, load_func in zip(self.func_paths, self.load_funcs):           
            if func_path.exists():
                load_func(func_path)  # load関数の実行
                
    def _save_funcs(self):
        """
        セーブ関数による保存
        """
        for func_path, save_func in zip(self.func_paths, self.save_funcs):
            save_func(func_path)  # save関数の実行
    
    def save_load_funcs(self, save_funcs, load_funcs, func_paths):
        """
        関数による保存についての設定と読み込み
        save_funcs: list of function
            保存する関数のリスト．各関数の引数はfunc_pathsの対応するパスとする．
        load_funcs: list of function
            ロードする関数のリスト．各関数の引数はfunc_pathsの対応するパスとする．
        func_paths: list of pathlib.Path
            保存・ロードするパスのリスト
        """
        arg_lists =[save_funcs, load_funcs, func_paths]
        
        assert all(list(map(lambda arg_list: isinstance(arg_list, list), arg_lists)))  # 皆list
        assert len(set(map(len , arg_lists)))==1  # 皆長さが一致
        
        self.save_funcs = save_funcs
        self.load_funcs = load_funcs
        self.func_paths = func_paths
        
        # 関数保存ファイルの読み込み
        self._load_funcs()
        
    def _write_tempfile(self):
        """
        一時ファイルの書き出し
        """
        with open(self.file_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["datetime", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
            writer.writerow(["start_count",self.counter])
            
    def save(self):
        """
        一時ファイル・保存ファイルを保存
        """
        self._write_tempfile()
        self._save_object()
        self._save_funcs()
        
    def remove_files(self):
        """
        利用した一時ファイル・保存ファイルをすべて削除する
        """
        # 一時ファイルの削除
        if self.file_path.exists():
            self.file_path.unlink()
        # 保存ファイルの削除
        if self.object_path is not None:
            if self.object_path.exists():
                self.object_path.unlink()
        for func_path in self.func_paths:
            if func_path.exists():
                # ディレクトリかファイルか判定
                if func_path.is_file():
                    func_path.unlink()
                elif func_path.is_dir():
                    shutil.rmtree(func_path)
    
    def _iter_finish(self):
        """
        イテレーションが終了したときの処理．親が存在するならばファイルを削除せず保存する．
        """
        if self.parent is None:  # ペアレントが無い場合
            self.remove_files()  # 一時ファイル・保存ファイルの削除
        else:  # ペアレントが存在する場合
            self.save()  # 一時ファイル・保存ファイルの保存
    
    def __call__(self, iterable):  # ジェネレーターを返す
        """
        for文にラップするための関数
        iterable: イテラブルなオブジェクト
        
        return
        ------
        ジェネレーター
        """
        iterable = iter(iterable)
        self.counter = 0  # カウンタの初期化
        while True:
            if self.counter < self.start_counter:
                self.counter += 1
                try:
                    next(iterable)  # 利用しない．進めるだけ
                except StopIteration:
                    self._iter_finish()
                    return None  # StopIterationで終了
                continue
            
            try:
                yield_item = next(iterable)  # iterableから一つ取得
                yield yield_item
            except StopIteration:
                self._iter_finish()
                return None  # StopIterationで終了
            
            self.counter += 1  # すべてが終了したら+1
            if self.each_save:  # 一時ファイルを指定回数ごとに保存
                if self.counter%self.save_span==0:
                    self.save()  # 一時ファイル・保存ファイルの保存
    
    def __enter__(self):
        return self
                
    def __exit__(self, ex_type, ex_value, trace):
        """
        with文が終了した際に，異常終了なら各ファイルを保存し，正常終了ならすでに処理してあるため何もしない
        """
        if ex_type is not None:  # エラーで終了した場合 
            self.save()
        return False

In [8]:
class CounterClosierThrough(CounterClosier):
    """
    CounterClosierを模した何もしないクロージャ．実装を変えずにtempfileの使用・不使用を切り替えるために利用する
    """
    def __call__(self, iterable):
        """
        ファイルを削除して，イテレータをそのまま返す
        """
        # このタイミングで，一時ファイル・保存ファイルを削除
        self.remove_files()
        # そのままイテレータを返す
        return iterable
    
    def __enter__(self):
        """
        何もしない
        """
        return self

    def __exit__(self, ex_type, ex_value, trace):
        """
        何もしない
        """
        return False

### ParentCounter 

In [9]:
class ParentCounter():
    """
    CounterClosierをまとめる親となるクラス
    """
    def __init__(self):
        self.child_counter_list = []
        
    def create_child(self, file_path, each_save=False, save_span=1, use_tempfile=True):
        """
        自身から子供を作成する
        file_path: pathlib.Path
            一時ファイルのパス
        each_save: bool
            指定回数ごとに保存するかどうか
        save_span: int, default:1
            指定価数ごとに保存する場合の，指定回数
        use_tempfile: bool
            一時ファイルを利用するかどうか
        """
        if use_tempfile:
            counter = CounterClosier(file_path, parent=self, each_save=each_save, save_span=save_span)
        else:
            counter = CounterClosierThrough(file_path, parent=self)
        
        self.child_counter_list.append(counter)
        return counter
        
    @staticmethod
    def create_non_parent_child(file_path, each_save=False, save_span=1, use_tempfile=True):
        """
        file_path: pathlib.Path
            一時ファイルのパス
        each_save: bool
            指定回数ごとに保存するかどうか
        save_span: int, default:1
            指定価数ごとに保存する場合の，指定回数
        use_tempfile: bool
            一時ファイルを利用するかどうか
        """
        if use_tempfile:
            counter = CounterClosier(file_path, parent=None, each_save=each_save, save_span=save_span)
        else:
            counter = CounterClosierThrough(file_path, parent=None)
        return counter
        
    def multi_child(self):
        """
        子供を複数まとめる場合にwith文で展開する
        """
        return MultiClosier(self)
    
    def all_close(self):
        """
        子の一時ファイル・保存ファイルを全て削除する．
        """
        [counter.remove_files() for counter in self.child_counter_list]

### インターフェースとなる関数 

In [10]:
def multi_count():
    """
    複数カウンタを作成するときに展開することで，一時ファイル・保存ファイルの削除をすべてが終了したタイミングで行うことができる．．
    """
    parent = ParentCounter()
    return parent.multi_child()


def enable_counter(file_path, use_tempfile=True, each_save=True, save_span=1):
    """
    for文をラップするCounterClosierオブジェクトを返す．with文で展開することで，エラーによる終了時に進捗状況(一時ファイル・保存ファイル)を保存する.
    with文に展開しなくても，each_saveをTrueにすることで，指定したイテレーション回数ごとに保存することもできる．
    
    file_path: pathlib.Path
        一時ファイルのパス
    use_tempfile: bool
        一時ファイル・保存ファイルを利用するかどうか．つまりこれをTrueにすると，利用しないのと全く同じになる
    each_save: bool
        指定回数ごとに保存するかどうか．
    save_span: int
        指定回数ごとに保存する場合の指定回数
    """
    if config.parent is None:  # グローバルのペアレントが存在しない場合
        counter = ParentCounter.create_non_parent_child(file_path, each_save=each_save, save_span=save_span, use_tempfile=use_tempfile)
        return counter
    else:  # グローバルのペアレントが存在する場合
        counter = config.parent.create_child(file_path, each_save=each_save, save_span=save_span, use_tempfile=use_tempfile)
        return counter
    
    
def simple_counter(file_path, iterable, use_tempfile=True, save_span=1):
    """
    for文をラップするジェネレータを直接返す．イテレーションの毎回で保存される．
    file_path: pathlib.Path
        一時ファイルのパス
    iterable: any of itrable
        イテラブルなオブジェクト
    use_tempfile: bool
        一時ファイル・保存ファイルを利用するかどうか．つまりこれをTrueにすると，利用しないのと全く同じになる
    save_span: int
        指定回数ごとに保存する場合の指定回数
    """
    return enable_counter(file_path, use_tempfile, each_save=True, save_span=save_span)(iterable)

### テストコード 

#### 一つの場合 

以下のように，`enable_counter`をwith文に添えた返り値(`CounterClosier`オブジェクト)でイテレーターをラップする．イテレーション内でエラーが生じた場合に，一時ファイルを保存し，次回はエラーが起きたイテレーションから再開できる．
この例では，iが4のときにKeybordInterruptを行った後，もう一度実行した結果である．

In [12]:
tempfile_path = Path("temp1.tmp")

with enable_counter(tempfile_path) as counter:
    for i in counter(range(10)):
        print(i)
        time.sleep(3)

4
5
6
7
8
9


#### 一つの場合(毎回保存する場合) 

with文を利用したくない場合，`enable_counter`の引数`each_save`をTrueにするか，`simple_couonter`が利用できる．どちらも異常終了時に一時ファイルを保存するわけではなく，イテレーションの指定回数ごとに保存する．また，`simple_counter`は直接ジェネレータを出力する．

In [14]:
tempfile_path = Path("temp2.tmp")

for i in simple_counter(tempfile_path, range(10)):
    print(i)
    time.sleep(3)

4
5
6
7
8
9


#### 二つ以上の場合 

`enable_counter`あるいは`simple_counter`のみでは，一つのfor文が終了したときに一時ファイルが削除されてしまうため，二つ以上for文が連続する場合に進捗を保存できない．`multi_count`を利用すればそのインデントブロックが終了するまで一時ファイルを残すことができる．以下の例では，一つ目のfor文が終了したのちにiが2の時点でKeybordInterruptを行い，再度実行した結果である

In [16]:
tempfile_path1 = Path("temp3.tmp")
tempfile_path2 = Path("temp4.tmp")

with multi_count():
    with enable_counter(tempfile_path1) as counter:
        for i in counter(range(10)):
            print("1:",i)
            time.sleep(3)
            
    print("1 is finished")
    for i in simple_counter(tempfile_path2, range(5)):
            print("2:",i)
            time.sleep(3)

1 is finished
2: 2
2: 3
2: 4


#### 再帰的に使う場合 

以下の例では，iが1,jが2の時にKeybordInterruptを行ったのち，再度実行したものである．

In [18]:
tempfile_path3 = Path("temp5.tmp")
tempfile_path4 = Path("temp6.tmp")

with enable_counter(tempfile_path3) as outer_counter:
    for i in outer_counter(range(3)):
        print("outer:",i)
        for j in simple_counter(tempfile_path4 ,range(5)):
                print("\tinner:",j)
                time.sleep(3)

outer: 1
	inner: 2
	inner: 3
	inner: 4
outer: 2
	inner: 0
	inner: 1
	inner: 2
	inner: 3
	inner: 4


#### オブジェクトの一時保存

イテレーションの進捗保存だけでなく，特定のオブジェクトも一時的に保存できる．その場合，以下のように`enable_counter`の返り値`CounterClosier`の`save_load_object`メソッドを利用できる．もちろん`save_load_object`はイテレーション内に記述
するべきではないが，withブロック内に記述する必要がある．登録したオブジェクトがイミュータブルな場合，イテレーション途中で`CounterClosier`の`object`プロパティを明示的に変更する．  

この例では，iが4のときにKeybordInterruptを行った後，もう一度実行した結果である．

In [20]:
tempfile_path = Path("temp7.tmp")

# 保存したいオブジェクト
save_object = {"sum":0}

with enable_counter(tempfile_path) as counter:
    # オブジェクトの登録(保存ファイルがある場合の読み込み)
    save_object = counter.save_load_object(save_object, Path("temp_sum.pickle"))
    print(save_object)
    for i in counter(range(10)):
        print("i:",i)
        time.sleep(3)
        
        save_object["sum"] += i
        # 変更を明示する場合，以下のようにする
        #counter.object = save_object
        
        print("sum:",save_object["sum"])

{'sum': 6}
i: 4
sum: 10
i: 5
sum: 15
i: 6
sum: 21
i: 7
sum: 28
i: 8
sum: 36
i: 9
sum: 45


毎回保存する場合，つまり`enable_counter`の引数`each_save`を`True`にした場合，with文を用いなくても保存できる．しかしイテレーション毎にpickleで保存するため，データの読み込み・書き出しのオーバーヘッドが加わることに注意する．`enable_counter`の引数`save_span`を指定することで，保存間隔を指定できる．

指定回数ごとに保存することによって，エラーで検知できないような終了(例えば，Google colabの接続切れなど)をしてしまっても，一次ファイルを保存できるメリットがある．

In [22]:
tempfile_path = Path("temp8.tmp")

# 保存したいオブジェクト
save_object = {"sum":0}

counter = enable_counter(tempfile_path, each_save=True, save_span=1)
# オブジェクトの登録(保存ファイルがある場合の読み込み)
save_object = counter.save_load_object(save_object, Path("temp_sum.pickle"))
print(save_object)

for i in counter(range(10)):
    print("i:",i)
    time.sleep(3)

    save_object["sum"] += i
    counter.object = save_object  # 一応明示的に変更

    print("sum:",save_object["sum"])

{'sum': 6}
i: 4
sum: 10
i: 5
sum: 15
i: 6
sum: 21
i: 7
sum: 28
i: 8
sum: 36
i: 9
sum: 45


#### 任意の保存・ロード関数の利用 

機械学習における重みファイルの保存など，オブジェクトの保存に外部の関数を利用したい場合がある．その場合は`CounterClosier`の`save_load_funcs`メソッドを利用できる．`save_load_funcs`の引数は`save_funcs`(保存用の関数のリスト),`load_funcs`(読み込み用の関数のリスト)，`func_paths`(二つの関数の引数となるパスのリスト)の3つのリストを対応するように渡す必要がある．保存用の関数・読み込み用の関数，はどちらもパスのみを引数とするため,任意の関数を利用する場合は無名関数などを用いて調節する必要がある．なお，`load_funcs`に与える関数は，保存したいオブジェクトをグローバル変数にして変更する必要があることに注意する．

In [21]:
import torch
import torch.nn as nn
import numpy as np

In [22]:
linear_model = nn.Linear(5, 10)
temp_array = np.zeros((2,2))
temp_tensor = torch.zeros((3,3))

# pickleでtensorを書き出す用の関数
def save_temp_tensor_as_pickle(save_path):
    with open(save_path, "wb") as f:
        pickle.dump(temp_tensor, f)
    
# 保存関数のリスト
save_funcs = [lambda save_path: torch.save(linear_model.state_dict(), save_path),
              lambda save_path: np.save(save_path, temp_array),
              save_temp_tensor_as_pickle
             ]

# pytorchのモデルを読み込む用の関数
def load_linear_model(load_path):
    global linear_model  # こちらの宣言は必要ない
    linear_model.load_state_dict(torch.load(load_path))
# ndarrayを読み込む用の関数
def load_temp_array(load_path):
    global temp_array  # 書き換えるため，グローバル変数宣言
    temp_array = np.load(load_path)
# pickleでtensorを読み込む用の関数
def load_temp_tensor(load_path):
    global temp_tensor  # 書き換えるため，グローバル変数宣言
    with open(load_path, "rb") as f:
        temp_tensor = pickle.load(f)

# ロード関数のリスト
load_funcs = [load_linear_model,
              load_temp_array,
              load_temp_tensor,
             ]

#　パスのリスト
func_paths = [Path("temp_linear_model.pth"),
              Path("temp_array.npy"),
              Path("temp_tensor.pickle")]

この例では，iが4のときにKeybordInterruptを行った後，もう一度実行した結果である．

In [24]:
tempfile_path = Path("temp9.tmp")

with enable_counter(tempfile_path) as counter:
    # 保存・ロード関数の登録(保存ファイルがある場合は読み込み)
    counter.save_load_funcs(save_funcs=save_funcs,
                            load_funcs=load_funcs,
                            func_paths=func_paths)
    
    print("load temp_array:", temp_array)
    print("load temp_tensor:", temp_tensor)
    for i in counter(range(10)):
        print("i:",i)
        time.sleep(3)
        
        temp_array += i * np.ones((2,2))
        temp_tensor += i * torch.ones((3,3))
        print("temp_array:", temp_array)
        print("temp_tensr:", temp_tensor)

load temp_array: [[6. 6.]
 [6. 6.]]
load temp_tensor: tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]])
i: 4
temp_array: [[10. 10.]
 [10. 10.]]
temp_tensr: tensor([[10., 10., 10.],
        [10., 10., 10.],
        [10., 10., 10.]])
i: 5
temp_array: [[15. 15.]
 [15. 15.]]
temp_tensr: tensor([[15., 15., 15.],
        [15., 15., 15.],
        [15., 15., 15.]])
i: 6
temp_array: [[21. 21.]
 [21. 21.]]
temp_tensr: tensor([[21., 21., 21.],
        [21., 21., 21.],
        [21., 21., 21.]])
i: 7
temp_array: [[28. 28.]
 [28. 28.]]
temp_tensr: tensor([[28., 28., 28.],
        [28., 28., 28.],
        [28., 28., 28.]])
i: 8
temp_array: [[36. 36.]
 [36. 36.]]
temp_tensr: tensor([[36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.]])
i: 9
temp_array: [[45. 45.]
 [45. 45.]]
temp_tensr: tensor([[45., 45., 45.],
        [45., 45., 45.],
        [45., 45., 45.]])


###  エラーとなる処理

以下のように，`multi_count`は再帰的に利用できない

In [56]:
with multi_count():
    with multi_count():
        pass

Exception: MultiCount has already opend. cannot open another MultiCount